In [ ]:
import numpy as np
from numpy import *
import pandas as pd
import h5py
import os
import re
import matplotlib.pyplot as plt
import plotly.express as px
import scipy.signal as signal
import scipy.io as sio
import scipy.stats
from scipy.fft import fft
import neurokit2 as nk
import heartpy as hp
import nolds
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
import warnings

In [ ]:
import sys
sys.path.append('/Users/jinyanwei/Desktop/BP_Model/Jinyw_code/')
from read_data import open_data
from filter_and_clean_data import clean_data
from segment_and_features import features_data
from random_forest import run_random_forest

all_data = open_data('/Users/jinyanwei/Desktop/BP_Model/Data/UCI/Part_1.mat')

In [ ]:
data_ready = pd.DataFrame()
for patient_number in range(1):
    if len(all_data[patient_number]) > 18000 :  # 5min-300beats-18000sample at least
        clean_data(all_data[patient_number], patient_number)
        features_data("/Users/jinyanwei/Desktop/BP_Model/Model_record/cleaned_data", patient_number)
        features_file = f'/Users/jinyanwei/Desktop/BP_Model/Model_record/features_data/Part1_feature{patient_number}.csv'
        if not os.path.exists(features_file):
            continue       
        data_ready = pd.concat([data_ready, pd.read_csv(features_file)])
run_random_forest(data_ready)

In [ ]:
## 小文件打开mat
import scipy.io as scio
import pandas as pd

data_path="/Users/jinyanwei/Desktop/BP_Model/Data/Cuffless_BP_Estimation/part_10.mat"
data = scio.loadmat(data_path)
len(data['p'][0])
all_patients = list(data['p'][0])
df = pd.DataFrame(all_patients[0]).T
df

In [ ]:
with h5py.File('/Users/jinyanwei/Desktop/BP_Model/Data/UCI/Part_1.mat', 'r') as file:
    print(list(file.keys()))
    references = np.array(file[list(file.keys())[1]]) #### Access a specific variable and convert it to a NumPy array   
    # Dereference the objects and store them in a list
    data = []
    for ref in references.flat:
        dereferenced_object = np.array(file[ref])
        data.append(dereferenced_object)  
#### Show the shape of the data
for i, array in enumerate(data):
    print(f"Shape of data[{i}]: {array.shape}")

In [ ]:
df = pd.DataFrame(data[0], columns=(('PPG','BP','ECG')))
df

In [ ]:
df = pd.DataFrame(data[0], columns=(('PPG','BP','ECG')))
ppg_data = df['PPG']
bp_data = df['BP']
ecg_data = df['ECG']

## Filter signals
sampling_rate = 125
ppg_filtered = nk.signal_filter(ppg_data, lowcut=0.5, highcut=50, method='butterworth', order=2, sampling_rate=sampling_rate)
ecg_filtered = nk.signal_filter(ecg_data, lowcut=0.5, highcut=50, method='butterworth', order=2, sampling_rate=sampling_rate)
filtered_df = df
filtered_df['PPG'], filtered_df['ECG'] = ppg_filtered, ecg_filtered
#### show the filtered PPG and ECG signals
ppg_df = pd.DataFrame(columns=(('PPG_original', 'PPG_filtered')))
ppg_df['PPG_original'] = ppg_data
ppg_df['PPG_filtered'] = ppg_filtered
display(ppg_df)
figPPG = px.line(ppg_df)
figPPG.show()
ecg_df = pd.DataFrame(columns=(('ECG_original', 'ECG_filtered')))
ecg_df['ECG_original'] = ecg_data
ecg_df['ECG_filtered'] = ecg_filtered
display(ecg_df)
figECG = px.line(ecg_df)
figECG.show()

## Clean data: make every beat only have one PPG/BP peak
#### Find ECG R-peaks
ecgpeaks, _ = signal.find_peaks(ecg_filtered, distance=sampling_rate//2.5)
print(ecgpeaks, len(ecgpeaks))
plt.figure(figsize=(50, 10))
for index in ecgpeaks:
    plt.scatter(index, ecg_filtered[index], marker="*")
plt.plot(ecg_filtered)
plt.show()

#### Make sure onle one PPG peak and one BP peak between two adjacent ECG R-peaks
cleaned_df = pd.DataFrame()
sum_beats = 0
times_recorder = []
for R_peak_number in range(len(ecgpeaks)-1):
    sum_beats += 1
    if ecg_filtered[ecgpeaks[R_peak_number]] > 0.2 and ecg_filtered[ecgpeaks[R_peak_number + 1]] > 0.2: # make sure the peaks are R-peak
        onebeat_ppgpeak, _ = signal.find_peaks(ppg_filtered[ecgpeaks[R_peak_number]:ecgpeaks[R_peak_number + 1]], distance = sampling_rate//2.5)
        onebeat_bppeak, _ = signal.find_peaks(bp_data[ecgpeaks[R_peak_number]:ecgpeaks[R_peak_number + 1]], distance = sampling_rate//2.5)
        if len(onebeat_ppgpeak) == 1 and ppg_filtered[ecgpeaks[R_peak_number] + onebeat_ppgpeak] > 0 and len(onebeat_bppeak) == 1 : # make sure only one BP signal for one beat 
            cleaned_df = pd.concat([cleaned_df, filtered_df[ecgpeaks[R_peak_number]:ecgpeaks[R_peak_number + 1]]])
            times_recorder.append(sum_beats)                

#### Show  original cleaned data:
print('All beats: ', sum_beats, ', Reserved beats: ', len(times_recorder))
display(cleaned_df)
figcleaned = px.line(cleaned_df)
figcleaned.show()

#### Reset indexs:
cleaned_df = cleaned_df.reset_index()
#### Show final cleaned data:
display(cleaned_df)
figcleaned = px.line(cleaned_df)
figcleaned.show()

## Segment signals from R-peak to get data of each beat.
df = cleaned_df
ppg_data = df['PPG']
bp_data = df['BP']
ecg_data = df['ECG']
sampling_rate = 125
ecgpeaks, _ = signal.find_peaks(ecg_data, distance=sampling_rate//2.5)
ppg_segments, ecg_segments, SBPlist, DBPlist, bplist = [], [], [], [], []
for peak_number in range(1, len(ecgpeaks)-2):    # data will be more stable from the second R-peak
    ppg_segments.append(ppg_data[ecgpeaks[peak_number]:ecgpeaks[peak_number + 1]].values)
    ecg_segments.append(ecg_data[ecgpeaks[peak_number]:ecgpeaks[peak_number + 1]].values)
    bplist = bp_data[ecgpeaks[peak_number]:ecgpeaks[peak_number+1]]
    SBPlist.append(max(bplist))
    DBPlist.append(min(bplist))
## Extract features of PPG and ECG.

#### Def features:
from numpy.fft import fft
def extract_ppg_features(signal):
    #### Time domain features
    mean = np.mean(signal)
    std_dev = np.std(signal)
    skewness = scipy.stats.skew(signal)
    kurtosis = scipy.stats.kurtosis(signal)
    #### Frequency domain features
    fft_values = fft(signal)
    power_spectrum = np.abs(fft_values)**2
    total_power = np.sum(power_spectrum)
    low_freq_power = np.sum(power_spectrum[:len(power_spectrum)//2]) / total_power
    high_freq_power = np.sum(power_spectrum[len(power_spectrum)//2:]) / total_power
    ppg_features = {
        'ppg_mean': mean,
        'ppg_std_dev': std_dev,
        'ppg_skewness': skewness,
        'ppg_kurtosis': kurtosis,
        'ppg_low_freq_power': low_freq_power,
        'ppg_high_freq_power': high_freq_power
    }
    return ppg_features

def extract_ecg_features(signal):
    # Time domain features
    mean = np.mean(signal)
    std_dev = np.std(signal)
    skewness = scipy.stats.skew(signal)
    kurtosis = scipy.stats.kurtosis(signal)
    # Frequency domain features
    fft_values = fft(signal)
    power_spectrum = np.abs(fft_values)**2
    total_power = np.sum(power_spectrum)
    low_freq_power = np.sum(power_spectrum[:len(power_spectrum)//2]) / total_power
    high_freq_power = np.sum(power_spectrum[len(power_spectrum)//2:]) / total_power
    ecg_features = {
        'ecg_mean': mean,
        'ecg_std_dev': std_dev,
        'ecg_skewness': skewness,
        'ecg_kurtosis': kurtosis,
        'ecg_low_freq_power': low_freq_power,
        'ecg_high_freq_power': high_freq_power
    }
    return ecg_features
#### Get features and save to csv:
ppg_feature_list = [extract_ppg_features(ppg_segment) for ppg_segment in ppg_segments]
ecg_feature_list = [extract_ecg_features(ecg_segment) for ecg_segment in ecg_segments]
features_df = pd.concat([pd.DataFrame(ppg_feature_list), pd.DataFrame(ecg_feature_list), pd.DataFrame({'SBP': SBPlist, 'DBP':DBPlist})], axis=1)
display(features_df)

#### Estimate BP
df = features_df
X_scaled = df.iloc[:, 1:-2].values
y_bp = df.iloc[:, -2:]

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_bp, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

result_bp = pd.DataFrame(columns=['SBP_actu','SBP_pred','DBP_actu','DBP_pred'])
result_bp['SBP_actu'] = y_test['SBP']
result_bp['DBP_actu'] = y_test['DBP']
result_bp['SBP_pred'] = y_pred[:,0]
result_bp['DBP_pred'] = y_pred[:,1]
result_bp = result_bp.sort_index()

rmse_sbp = metrics.mean_squared_error(result_bp['SBP_actu'], result_bp['SBP_pred'])**0.5
rmse_dbp = metrics.mean_squared_error(result_bp['DBP_actu'], result_bp['DBP_pred'])**0.5
mae_sbp = metrics.mean_absolute_error(result_bp['SBP_actu'], result_bp['SBP_pred'])
mae_dbp = metrics.mean_absolute_error(result_bp['DBP_actu'], result_bp['DBP_pred'])

print(f'{len(y_train)} beats to train and {len(y_test)} beats to test.', 
    '\n'f"Root mean squared error for SBP: {rmse_sbp:.3f}", 
    '\n'f"Root mean squared error for DBP: {rmse_dbp:.3f}", 
    '\n'f"Mean absolute error for SBP: {mae_sbp:.3f}", 
    '\n'f"Mean absolute error for DBP: {mae_dbp:.3f}"
    )
def write_record_totxt():
    file_handle = open('/Users/jinyanwei/Desktop/BP_Model/Model_record/random_forest_result.txt', mode='a')
    file_handle.write(f'{len(y_train)} beats to train and {len(y_test)} beats to test.')
    file_handle.write('\n')
    file_handle.write(f"Root mean squared error for SBP: {rmse_sbp:.3f}")
    file_handle.write('\n')
    file_handle.write(f"Root mean squared error for DBP: {rmse_dbp:.3f}")
    file_handle.write('\n')
    file_handle.write(f"Mean absolute error for SBP: {mae_sbp:.3f}")
    file_handle.write('\n')
    file_handle.write(f"Mean absolute error for DBP: {mae_dbp:.3f}")
    file_handle.write('\n')
    file_handle.write('\n')
    return


#### Draw pictures

plt.figure(figsize=(30, 15))
plt.subplot(2, 1, 1)
x1 = np.array((range(len(result_bp))))
sbp1 = np.array(result_bp['SBP_pred'])
sbp2 = np.array(result_bp['SBP_actu'])
plt.plot(x1, sbp1, label='SBP_pred')
plt.plot(x1, sbp2, label='SBP_actu')
plt.title('SBP')

plt.subplot(2, 1, 2)
x2 = x1
dbp1 = np.array(result_bp['DBP_pred'])
dbp2 = np.array(result_bp['DBP_actu'])
plt.plot(x2, dbp1, label='DBP_pred')
plt.plot(x2, dbp2, label='DBP_actu')
plt.title('DBP')
plt.legend()
plt.show()

